# Generative Agents in LangChain

This notebook implements a generative agent based on the paper [Generative Agents: Interactive Simulacra of Human Behavior](https://arxiv.org/abs/2304.03442) by Park, et. al.

In it, we leverage a time-weighted Memory object backed by a LangChain Retriever.

In [1]:
import os

os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [2]:
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain.retrievers import TimeWeightedVectorStoreRetriever
from langchain_community.vectorstores import FAISS
from langchain_community.chat_models import ChatOllama
from langchain_community.embeddings import OllamaEmbeddings
from termcolor import colored

In [3]:
USER_NAME = "Pam"  # The name you want to use when interviewing the agent.

# Local LLM
LLM = ChatOllama(
    model="qwen2",
    keep_alive=-1, # keep the model loaded indefinitely
    temperature=0,
    max_new_tokens=4096 # officailly 4096
    )

### Generative Agent Memory Components

This tutorial highlights the memory of generative agents and its impact on their behavior. The memory varies from standard LangChain Chat memory in two aspects:

1. **Memory Formation**

   Generative Agents have extended memories, stored in a single stream:
      1. Observations - from dialogues or interactions with the virtual world, about self or others
      2. Reflections - resurfaced and summarized core memories


2. **Memory Recall**

   Memories are retrieved using a weighted sum of salience, recency, and importance.

You can review the definitions of the `GenerativeAgent` and `GenerativeAgentMemory` in the [reference documentation]("https://api.python.langchain.com/en/latest/modules/experimental.html") for the following imports, focusing on `add_memory` and `summarize_related_memories` methods.

In [4]:
from langchain_experimental.generative_agents import (
    GenerativeAgent,
    GenerativeAgentMemory,
)

## Memory Lifecycle

Summarizing the key methods in the above: `add_memory` and `summarize_related_memories`.

When an agent makes an observation, it stores the memory:
    
1. Language model scores the memory's importance (1 for mundane, 10 for poignant)
2. Observation and importance are stored within a document by TimeWeightedVectorStoreRetriever, with a `last_accessed_time`.

When an agent responds to an observation:

1. Generates query(s) for retriever, which fetches documents based on salience, recency, and importance.
2. Summarizes the retrieved information
3. Updates the `last_accessed_time` for the used documents.


## Create a Generative Character



Now that we've walked through the definition, we will create two characters named "Tommie" and "Eve".

In [5]:
import numpy as np

import faiss


def score_normalizer(val: float) -> float:
    ret = 1.0 - 1.0 / (1.0 + np.exp(val))
    #print("val: "+str(float(val))+"_"+"ret: "+str(ret))
    return ret

def create_new_memory_retriever():
    """Create a new vector store retriever unique to the agent."""
    # Define your embedding model
    embeddings_model = OllamaEmbeddings(model="qwen2")

    # Automatically determine the size of the embeddings
    test_embedding = embeddings_model.embed_query("test query")
    embedding_size = len(test_embedding)
    
    # Initialize the vectorstore as empty
    index = faiss.IndexFlatL2(embedding_size)
    
    # Initialize FAISS vector store
    vectorstore = FAISS(
        embeddings_model,
        index,
        InMemoryDocstore({}),
        {},
        relevance_score_fn=score_normalizer,
        normalize_L2=True
    )
    
    # Create and return the retriever
    return TimeWeightedVectorStoreRetriever(
        vectorstore=vectorstore, 
        other_score_keys=["importance"], 
        k=15
    )

In [6]:
tommies_memory = GenerativeAgentMemory(
    llm=LLM,
    memory_retriever=create_new_memory_retriever(),
    verbose=False,
    reflection_threshold=8,  # we will give this a relatively low number to show how reflection works
)

tommie = GenerativeAgent(
    name="Tommie",
    age=25,
    traits="anxious, likes design, talkative",  # You can add more persistent traits here
    status="looking for a job",  # When connected to a virtual world, we can have the characters update their status
    llm=LLM,
    memory=tommies_memory,
)

In [7]:
# The current "Summary" of a character can't be made because the agent hasn't made
# any observations yet.
print(tommie.get_summary())

Name: Tommie (age: 25)
Innate traits: anxious, likes design, talkative
Tommie is characterized by being straightforward, honest, and dependable. He values integrity and reliability in his interactions and commitments.


In [8]:
# We can add memories directly to the memory object
tommie_observations = [
    "Tommie remembers his dog, Bruno, from when he was a kid",
    "Tommie feels tired from driving so far",
    "Tommie sees the new home",
    "The new neighbors have a cat",
    "The road is noisy at night",
    "Tommie is hungry",
    "Tommie tries to get some rest.",
]

for observation in tommie_observations:
    tommie.memory.add_memory(observation)

In [9]:
# Now that Tommie has 'memories', their self-summary is more descriptive, though still rudimentary.
# We will see how this summary updates after more observations to create a more rich description.
print(tommie.get_summary(force_refresh=True))

Name: Tommie (age: 25)
Innate traits: anxious, likes design, talkative
Tommie appears to be someone who has recently moved into a new home, likely after a long journey as he feels tired from driving far. He seems to have a nostalgic connection with dogs, possibly remembering his childhood pet, Bruno. Tommie values rest and might be sensitive to noise at night, as indicated by the mention of a noisy road. Additionally, he is experiencing hunger, suggesting that he may need to address his basic needs soon.
